## Modules à importer

In [1]:
import math
import scipy
import ztfidr
import pickle
import numpy as np
import pandas as pd
import iminuit as im
from ztfidr import io
from ztfidr.sample import Sample
import ipywidgets as ipw
from variaIa import tools_ar
#from variaIa import tools_ar_sc
from importlib import reload
from variaIa import stretchevol
from variaIa.tools_ar import surveys, df_cons, df_nc, df_c, df_full, z_max
#from variaIa.tools_ar_sc import surveys, df_cons, df_nc, df_c, df_full, z_max
from IPython.display import display, Latex
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
from mpl_toolkits import mplot3d
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "EB Garamond",
    "axes.labelsize": 'x-large',
    "xtick.labelsize": 'x-large',
    "ytick.labelsize": 'x-large'})
import random

FileNotFoundError: [Errno 2] No such file or directory: '../../../Data/sne/ancillary_g10_n.FITRES'

In [42]:
def load_clean_data(red_min=0, red_max=0.055,
                    x1_min=-3, x1_max=3,
                    c_min=-0.3, c_max=0.3,
                    x1_err_max=1, c_err_max=0.3,
                    fitprob_min=1e-25):
    sample = Sample.load()
    sample.load_phasedf()
    salt = sample.get_data(goodcoverage=True).reset_index().rename({'index': 'ztfname'},axis=1)
    hostmags = io.get_host_mags()
    localhost = hostmags.xs('2kpc', axis=1)
    globalhost = hostmags.xs('global', axis=1)
    dlr = globalhost['host_dlr']
    dlr[dlr<0] = pd.NA
    localhost = localhost.assign(dlr=np.array(dlr))
    #salt.rename(columns={‘z’: ‘redshift_z’}, inplace=True)
    df = salt.merge(localhost, how='outer', on='ztfname')
    g = df['g']
    g[g<-90] = pd.NA
    df['g'] = g
    r = df['r']
    r[r<-90] = pd.NA
    df['r'] = r
    z = df['z']
    z[z<-90] = pd.NA
    df['z'] = z
    z_err = df['z_err']
    z_err[z_err>80] = pd.NA
    df['z_err'] = z_err
    g_err = df['g_err']
    g_err[g_err>80] = pd.NA
    df['g_err'] = g_err
    r_err = df['r_err']
    r_err[r_err>80] = pd.NA
    df['r_err'] = r_err
    #dlr = io.get_host_data()[‘host_dlr’]
    #dlr[dlr<0] = pd.NA
    #df[‘dlr’] = dlr
    data = df.loc[df['x1'].between(x1_min, x1_max) & df['c'].between(c_min, c_max)
                  & (df['x1_err']<x1_err_max) & (df['c_err']<c_err_max)
                  & (df['fitprob']>fitprob_min)]
    #data = data[data[‘z_quality’].isin(redshift_quality)]
    data = data.assign(color=data['g']-data['z'])
    data = data.assign(color_err=np.sqrt(data['g_err']**2 + data['z_err']**2))
    data = data.assign(mag=-2.5*np.log10(data['x0']) + 25)
    data = data.assign(mag_err=abs(2.5*data['x0_err']/(np.log(10)*data['x0'])))
    return data

In [43]:
ztdf = load_clean_data()

/home/nicolas/.installs/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/home/nicolas/.installs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/nicolas/.installs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/h

In [47]:
ztdf.to_csv('../../../Data/sne/ztf_full.csv', index=False)

In [50]:
ztdf.columns

Index(['ztfname', 'redshift', 'redshift_err', 'redshift_source',
       'redshift_salt', 't0', 'x0', 'x1', 'c', 'mwebv', 'mwr_v', 't0_err',
       'x0_err', 'x1_err', 'c_err', 'cov_t_x0', 'cov_t_x1', 'cov_t_c',
       'cov_x0_x1', 'cov_x0_c', 'cov_x1_c', 'fitprob', 'lccoverage_flag',
       'lcquality_flag', 'frac_fitted', 'ra', 'dec', 'coord_source',
       'iau_name', 't0day', 'g', 'r', 'i', 'z', 'y', 'g_err', 'r_err', 'i_err',
       'z_err', 'y_err', 'dlr', 'color', 'color_err', 'mag', 'mag_err'],
      dtype='object')

In [20]:
ztdf.to_csv('../../../Data/sne/ztf_full_complete.csv')

In [16]:
ztdf.rename({'redshift': 'redshifts',
             'x1': 'stretchs', 'x1_err': 'stretchs_err',
             'c': 'colors', 'c_err': 'colors_err'},
            axis=1, inplace=True)

In [18]:
ztdf

,ztfname,redshifts,redshift_err,redshift_source,redshift_salt,t0,x0,stretchs,colors,mwebv,...,g_err,r_err,i_err,z_err,y_err,dlr,color,color_err,mag,mag_err
5,ZTF18aadzfso,0.050000,0.00500,bts_private,0.050000,58888.326595,0.000812,-0.271364,0.157224,0.087386,...,0.020409,0.020197,0.020052,0.020204,0.021040,0.018130,1.482393,0.028718,32.726261,0.037387
11,ZTF18aahfbqp,0.041145,0.00002,host,0.041145,59208.886022,0.003025,-1.954567,-0.098493,0.019829,...,0.020071,0.020042,0.020020,0.020039,0.020049,0.026559,1.369164,0.028362,31.298270,0.058778
15,ZTF18aahheaj,0.052720,0.00002,host,0.052720,58226.020748,0.001421,0.853012,0.000986,0.021771,...,0.097145,0.061807,0.068938,0.085836,0.129176,0.530336,0.573816,0.129634,32.118858,0.038117
24,ZTF18aahtjsc,0.052933,0.00002,host,0.052933,58576.539890,0.000708,-0.003081,0.215816,0.014656,...,0.020434,0.020169,0.020067,0.020214,0.020473,0.117854,1.348735,0.028743,32.875575,0.039224
25,ZTF18aahvndq,0.015439,0.00002,host,0.015439,58991.711078,0.012460,-0.992664,0.081165,0.010925,...,0.020400,0.020114,0.020049,0.020096,0.020175,0.057390,1.431878,0.028636,29.761185,0.038971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2958,ZTF20adadsgm,0.043710,0.00002,host,0.043710,59218.850731,0.001786,-0.907218,0.019987,0.049054,...,0.022266,0.021806,0.020506,0.021752,0.024242,0.392645,1.240783,0.031128,31.870590,0.033063
2959,ZTF20adadshh,0.024682,0.00002,host,0.024682,59220.908116,0.003165,-0.759236,0.240956,0.052729,...,0.020040,0.020024,0.020018,0.020030,0.020079,0.162284,1.245940,0.028334,31.249071,0.037697
2960,ZTF20adagenq,0.024389,0.00002,host,0.024389,59225.181034,0.007494,0.323592,-0.062301,0.014554,...,0.022632,0.020932,0.020624,0.021313,0.023374,0.551259,1.225271,0.031087,30.313261,0.034577
2962,ZTF21aaaajfe,0.050000,0.00500,tns_sn,0.050000,59221.866040,0.001673,-1.833140,-0.009533,0.028439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.941257,0.034794


In [33]:
df_ztf = ztdf.drop(columns=['redshift_err', 'redshift_source', 'redshift_salt',
                            't0', 'x0', 'mwebv', 'mwr_v',
                            't0_err', 'x0_err',
                            'cov_t_x0', 'cov_t_x1', 'cov_t_c',
                            'cov_x0_x1', 'cov_x0_c', 'cov_x1_c',
                            'fitprob', 'lccoverage_flag', 'lcquality_flag', 'frac_fitted',
                            'ra', 'dec', 'coord_source', 'iau_name', 't0day',
                            'g', 'r', 'i', 'z', 'y',
                            'g_err', 'r_err', 'i_err', 'z_err', 'y_err',
                            'dlr', 'mag', 'mag_err'])
df_ztf.rename(columns={'ztfname': 'survey'}, inplace=True)
df_ztf['survey'] = ['ZTF' for i in range(len(df_ztf))]
df_ztf['hostmass'] = [pd.NA for i in range(len(df_ztf))]
df_ztf['hostmass_err'] = [pd.NA for i in range(len(df_ztf))]
df_ztf['infor'] = [stretchevol.Evol3G2M2S.delta(z) for z in df_ztf.redshifts]
df_ztf['py'] = [pd.NA for i in range(len(df_ztf))]
df_ztf['lssfr'] = [pd.NA for i in range(len(df_ztf))]
df_ztf['lssfr_err_d'] = [pd.NA for i in range(len(df_ztf))]
df_ztf['lssfr_err_u'] = [pd.NA for i in range(len(df_ztf))]

In [34]:
df_ztf

,survey,redshifts,stretchs,colors,stretchs_err,colors_err,color,color_err
5,ZTF,0.050000,-0.271364,0.157224,0.135631,0.030401,1.482393,0.028718
11,ZTF,0.041145,-1.954567,-0.098493,0.251621,0.051371,1.369164,0.028362
15,ZTF,0.052720,0.853012,0.000986,0.122863,0.031435,0.573816,0.129634
24,ZTF,0.052933,-0.003081,0.215816,0.164934,0.033235,1.348735,0.028743
25,ZTF,0.015439,-0.992664,0.081165,0.083858,0.032437,1.431878,0.028636
...,...,...,...,...,...,...,...,...
2958,ZTF,0.043710,-0.907218,0.019987,0.078024,0.027916,1.240783,0.031128
2959,ZTF,0.024682,-0.759236,0.240956,0.075043,0.031156,1.245940,0.028334
2960,ZTF,0.024389,0.323592,-0.062301,0.089957,0.029666,1.225271,0.031087
2962,ZTF,0.050000,-1.833140,-0.009533,0.114876,0.031052,NaN,NaN


In [10]:
df_nc.columns

Index(['survey', 'stretchs', 'stretchs_err', 'colors', 'colors_err',
       'hostmass', 'hostmass_err', 'redshifts', 'infor', 'py', 'lssfr',
       'lssfr_err_d', 'lssfr_err_u'],
      dtype='object')

In [11]:
df_nc.loc[df_nc['survey'] == 'PS1']

,survey,stretchs,stretchs_err,colors,colors_err,hostmass,hostmass_err,redshifts,infor,py,lssfr,lssfr_err_d,lssfr_err_u
281,PS1,0.036512,0.414800,0.136000,0.036563,10.793,0.0550,0.2305,0.608620,0.0,0.0,0.0,0.0
282,PS1,0.969200,0.355300,-0.109525,0.025322,8.938,0.6690,0.2534,0.620847,0.0,0.0,0.0,0.0
283,PS1,-2.348800,0.126500,-0.012955,0.028660,6.593,0.1180,0.1526,0.564242,0.0,0.0,0.0,0.0
284,PS1,0.091133,0.098665,0.012847,0.025693,9.799,0.5200,0.1023,0.533314,0.0,0.0,0.0,0.0
285,PS1,-0.204200,0.142200,-0.072658,0.025888,10.902,0.0090,0.1959,0.589441,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,PS1,1.774400,0.624200,-0.093497,0.035101,9.499,0.1450,0.3018,0.645481,0.0,0.0,0.0,0.0
437,PS1,0.410800,0.491300,0.088155,0.061195,10.819,0.1900,0.2902,0.639725,0.0,0.0,0.0,0.0
438,PS1,-0.729905,0.254800,-0.029056,0.034745,11.543,0.0160,0.3092,0.649104,0.0,0.0,0.0,0.0
439,PS1,-0.017351,0.305100,-0.058879,0.035811,9.851,0.1865,0.2956,0.642416,0.0,0.0,0.0,0.0
